In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *


#clasificadores
from utils.clasificacion import *

#otros
import os
import numpy as np
import pandas as pd
import timeit

#variables para guardar los resultados
tipo = pruebas[0]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

In [3]:
print tipo
print dataset_name
print dims
print data_path

baseline
amazon
3000
data


## Pruebas con el dataset Amazon

#### El formato de cada prueba es:

* nombre_prueba (parámetro=valor)
 
#### Por ejemplo, para la prueba de mSDA con 5 capas, el nombre es:

* mSDA (l=5)

#### Las pruebas para cada método son:

* SDA: Cantidad de capas entre {1, 3, 5} con la mitad de la dimensionalidad original por cada capa y cantidad de epochs entre {25, 50}.
* mSDA: Cantidad de capas entre {1, 3, 5}.
* PCA: Ejecuciones cambiando la dimensionalidad a 1/2 y 1/4 de la original.
* GFK: Pruebas con cantidad de sub-dimensiones entre {10, 50, 100}


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# 5000 ejemplos - 3000 dimensiones
dims = 300
X = np.random.randint(0, 2, size = (500, dims))
X.shape

(500, 300)

## GFK

In [6]:
from utils.adaptacion import adapt_gfk

Using Theano backend.
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [7]:
new_times_path = os.path.join(times_path, "gfk.csv")

if os.path.exists(new_times_path):
    df_gfk = pd.read_csv(new_times_path)
    i = df_gfk.shape[0]
else:
    df_gfk = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [ ]:
size_dims = int(dims/2)
n_subs = [10, 50, 100]

tgt = domains[0]
src = domains[1]

X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

In [ ]:
runs = 5

for j in xrange(runs):
    print j+1
    for n in n_subs:
        t_inicio = timeit.default_timer()

        nombre = "GFK (n_dims=%d)" % n
        print nombre



        t_fin = timeit.default_timer()
        tiempo = (t_fin-t_inicio)/60.

        print "\tEntrenado en %.3fm" % tiempo

        df_gfk.loc[i] = [nombre, tiempo]
        i = i+1

In [8]:
df_gfk = df_gfk.groupby('Adaptacion').mean().reset_index()
df_gfk

,Adaptacion,Unnamed: 0,Tiempo
0,GFK (n_dims=10),6,0.000003
1,GFK (n_dims=100),8,0.000017
2,GFK (n_dims=50),7,0.000002


In [9]:
print "Guardando en %s" % new_times_path
df_gfk.to_csv(new_times_path)
print "Resultados guardados."

Guardando en tiempos/gfk.csv
Resultados guardados.


# mSDA Theano

In [34]:
from mSDA.msda_theano import mSDATheano
import theano.tensor as T

In [35]:
new_times_path = os.path.join(times_path, "msda.csv")

if os.path.exists(new_times_path):
    df_msda = pd.read_csv(new_times_path)
    i = df_msda.shape[0]
else:
    df_msda = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [36]:
layers = [1, 3, 5]
x = T.dmatrix('x')
runs = 5

In [37]:
for j in xrange(runs):
    print j+1
    for layer in layers:
        nombre = "mSDA (l=%d)" % layer
        print nombre

        msda = mSDATheano(x, layer, 0.3)
        tiempo = msda.fit(X)

        df_msda.loc[i] = [nombre, tiempo]

        i = i+1

1
mSDA (l=1)
	Entrenando capa  1
	Entrenado en 0.01m

mSDA (l=3)
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.01m

mSDA (l=5)
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 0.02m

2
mSDA (l=1)
	Entrenando capa  1
	Entrenado en 0.00m

mSDA (l=3)
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.01m

mSDA (l=5)
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 0.02m

3
mSDA (l=1)
	Entrenando capa  1
	Entrenado en 0.00m

mSDA (l=3)
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.01m

mSDA (l=5)
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 0.02m

4
mSDA (l=1)
	Entrenando capa  1
	Entrenado en 0.01m

mSDA (l=3)
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.01m

mSDA (l=5)
	Entrenan

In [38]:
df_msda = df_msda.groupby('Adaptacion').mean().reset_index()
df_msda

In [39]:
print "Guardando en %s" % new_times_path
df_msda.to_csv(new_times_path)
print "Resultados guardados."

Guardando en tiempos/msda.csv
Resultados guardados.


# SDA

In [6]:
from utils.adaptacion import create_SDA
from sklearn.model_selection import train_test_split

Using Theano backend.
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [11]:
new_times_path = os.path.join(times_path, "sda.csv")

if os.path.exists(new_times_path):
    df_sda = pd.read_csv(new_times_path)
    i = df_sda.shape[0]
else:
    df_sda = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [8]:
# se divide el dataset para los datos de entrenamiento y validacion del SDA
X_train, X_val, _, _ = train_test_split(X, np.zeros(X.shape[0]), test_size=0.2, random_state=42)

print X_train.shape
print X_val.shape

(400, 300)
(100, 300)


In [9]:
layers = [[dims/2],
          [dims/2, dims/2, dims/2],
          [dims/2, dims/2, dims/2, dims/2, dims/2]]

epochs = [25, 50]
runs = 5

In [12]:
for j in xrange(runs):
    print j+1
    for epoch in epochs:
        for layer in layers:
            nombre = "SDA (l=%d, epochs=%d)" % (len(layer), epoch)
            print nombre

            autoencoder, encoder = create_SDA(X_train.shape[1], layer, 0.3)

            t_inicio = timeit.default_timer()

            print "\tEntrenando autoencoder..."
            autoencoder.fit(X_train, X_train,
               epochs=epoch,
               batch_size=256,
               shuffle=True,
               verbose=0,
               validation_data=(X_val, X_val))

            t_fin = timeit.default_timer()

            tiempo = (t_fin - t_inicio)/60.
            print "\tEntrenado en %.3fm" % tiempo

            df_sda.loc[i] = [nombre, tiempo]

            i = i+1

1
SDA (l=1, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.026m
SDA (l=3, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.042m
SDA (l=5, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.055m
SDA (l=1, epochs=50)
	Entrenando autoencoder...
	Entrenado en 0.032m
SDA (l=3, epochs=50)
	Entrenando autoencoder...
	Entrenado en 0.053m
SDA (l=5, epochs=50)
	Entrenando autoencoder...
	Entrenado en 0.074m
2
SDA (l=1, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.023m
SDA (l=3, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.037m
SDA (l=5, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.054m
SDA (l=1, epochs=50)
	Entrenando autoencoder...
	Entrenado en 0.029m
SDA (l=3, epochs=50)
	Entrenando autoencoder...
	Entrenado en 0.053m
SDA (l=5, epochs=50)
	Entrenando autoencoder...
	Entrenado en 0.090m
3
SDA (l=1, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.026m
SDA (l=3, epochs=25)
	Entrenando autoencoder...
	Entrenado en 0.041m
SDA (l=5, epochs=25)
	Entren

In [13]:
df_sda = df_sda.groupby('Adaptacion').mean().reset_index()
df_sda

,Adaptacion,Tiempo
0,"SDA (l=1, epochs=25)",0.024449
1,"SDA (l=1, epochs=50)",0.032180
2,"SDA (l=3, epochs=25)",0.039831
3,"SDA (l=3, epochs=50)",0.058598
4,"SDA (l=5, epochs=25)",0.056348
5,"SDA (l=5, epochs=50)",0.085875


In [14]:
print "Guardando en %s" % new_times_path
df_sda.to_csv(new_times_path)
print "Resultados guardados."

Guardando en tiempos/sda.csv
Resultados guardados.


# PCA

In [47]:
from sklearn.decomposition import PCA

In [57]:
new_times_path = os.path.join(times_path, "pca.csv")

if os.path.exists(new_times_path):
    df_pca = pd.read_csv(new_times_path)
    i = df_pca.shape[0]
else:
    df_pca = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [49]:
n_components = [dims/2, dims/4]
runs = 5

In [62]:
for j in xrange(5):
    print j+1
    for n in n_components:
        nombre = "PCA (n=%d)" % n
        print "\t%s" % nombre

        new_model = PCA(n_components=n)
        
        t_inicio = timeit.default_timer()
        new_model.fit(X)
        t_fin = timeit.default_timer()

        tiempo = (t_fin - t_inicio)/60.
        print "\tEntrenado en %.3fm" % tiempo
        
        df_pca.loc[i] = [nombre, tiempo]
        i = i+1

1
	PCA (n=150)
	Entrenado en 0.001m
	PCA (n=75)
	Entrenado en 0.001m
2
	PCA (n=150)
	Entrenado en 0.001m
	PCA (n=75)
	Entrenado en 0.001m
3
	PCA (n=150)
	Entrenado en 0.001m
	PCA (n=75)
	Entrenado en 0.001m
4
	PCA (n=150)
	Entrenado en 0.001m
	PCA (n=75)
	Entrenado en 0.001m
5
	PCA (n=150)
	Entrenado en 0.001m
	PCA (n=75)
	Entrenado en 0.001m


In [64]:
df_pca = df_pca.groupby('Adaptacion').mean().reset_index()
df_pca

,Adaptacion,Tiempo
0,PCA (n=150),0.000638
1,PCA (n=75),0.000606


In [65]:
print "Guardando en %s" % new_times_path
df_pca.to_csv(new_times_path)
print "Resultados guardados."

Guardando en tiempos/pca.csv
Resultados guardados.
